In [ ]:
import CSV
import DataFrames
import Dates
import JLD2
import Memoize: @memoize
import Setfield
import Setfield: @set

## Static data

In [ ]:
function get_data_path(file)
    joinpath(@__DIR__, "../../data/$file")
end;

In [ ]:
function read_csv(x; kw...)
    CSV.read(x, DataFrames.DataFrame; types = String, missingstring = nothing, kw...)
end;

In [ ]:
@memoize function num_users(dataset)
    df = read_csv(get_data_path("processed_data/$dataset.relabel_userid_map.csv"))
    maximum(parse.(Int32, df.userid))
end

@memoize function num_items(medium)
    df = read_csv(get_data_path("processed_data/item_counts.csv"))
    parse(Int32, first(df[:, medium]))
end;

In [ ]:
@memoize function get_status(status::Symbol)::Int32
    df = read_csv(get_data_path("processed_data/status.csv"))
    status_encoding =
        Dict(Symbol(k) => parse(Int32, v) for (k, v) in zip(df.name, df.encoding))
    status_encoding[status]
end;

In [ ]:
@memoize function get_unix_timestamp(dataset::String, col::Symbol)
    df = read_csv(get_data_path("processed_data/$dataset.timestamps.csv"))
    parse(Int64, first(df[:, col]))
end

function get_timestamp(x::Dates.Second)
    Dates.value(x) /
    (get_unix_timestamp("training", :max_ts) - get_unix_timestamp("training", :min_ts))
end

function get_timestamp(x::Dates.Day)
    get_timestamp(convert(Dates.Second, x))
end;

@memoize function get_timestamp(dataset::String, col::Symbol)
    get_timestamp(
        Dates.Second(
            get_unix_timestamp(dataset, col) - get_unix_timestamp("training", :min_ts),
        ),
    )
end;

## Reading and writing data

In [ ]:
@kwdef struct RatingsDataset
    source::Vector{Int32} = []
    medium::Vector{Int32} = []
    userid::Vector{Int32} = []
    itemid::Vector{Int32} = []
    status::Vector{Int32} = []
    rating::Vector{Float32} = []
    updated_at::Vector{Float64} = []
    created_at::Vector{Float64} = []
    started_at::Vector{Float64} = []
    finished_at::Vector{Float64} = []
    update_order::Vector{Int32} = []
    progress::Vector{Float32} = []
    progress_volumes::Vector{Float32} = []
    repeat_count::Vector{Int32} = []
    priority::Vector{Int32} = []
    sentiment::Vector{Int32} = []
    metric::Vector{Float32} = []
end

function RatingsDataset(d::Dict)
    cols = string.(fieldnames(RatingsDataset)[1:end-1])
    RatingsDataset([d[c] for c in cols]..., [])
end

function subset(x::RatingsDataset, ord)
    mask(arr) = !isempty(arr) ? arr[ord] : arr
    RatingsDataset([mask(getfield(x, c)) for c in fieldnames(RatingsDataset)]...)
end

function cat(x::RatingsDataset, y::RatingsDataset)
    nonempty(df, f) = length(getfield(x, f)) != 0
    for f in fieldnames(RatingsDataset)
        @assert nonempty(x, f) == nonempty(y, f) "cat: missing field $field"
    end
    RatingsDataset(
        [vcat(getfield(x, c), getfield(y, c)) for c in fieldnames(RatingsDataset)]...,
    )
end;

In [ ]:
function get_datasets()
    outdir = get_data_path("splits")
    available = ispath(outdir) ? readdir(outdir) : String[]
    possible = ["training", "streaming", "test", "causal"]
    @assert Set(available) ⊆ Set(possible)
    [x for x in possible if x in available]
end

const ALL_DATASETS = get_datasets()
const ALL_SPLITS = ["train", "test_input", "test_output"]
const ALL_MEDIUMS = ["manga", "anime"]
const ALL_METRICS = ["rating", "watch", "plantowatch", "drop"];

In [ ]:
function get_split(dataset::String, split::String, medium::String, fields::Vector{Symbol})
    @assert dataset in ALL_DATASETS
    @assert split in ALL_SPLITS
    @assert medium in ALL_MEDIUMS
    @assert Set(fields) ⊆ Set(fieldnames(RatingsDataset))
    df = RatingsDataset()
    fn = get_data_path("splits/$dataset/$split")
    for i = 1:length(fields)
        df = Setfield.set(
            df,
            Setfield.PropertyLens{fields[i]}(),
            JLD2.load("$fn/$(fields[i]).jld2", medium),
        )
    end
    @assert [length(getfield(df, f)) for f in fields] |> Set |> length <= 1
    df
end

function as_metric(df::RatingsDataset, metric::String)
    @assert metric in ALL_METRICS
    if metric == "rating"
        df = subset(df, df.rating .!= 0)
        df = @set df.metric = copy(df.rating)
    elseif metric == "watch"
        df = subset(df, df.status .> get_status(:planned))
        df = @set df.metric = ones(Float32, length(df.status))
    elseif metric == "plantowatch"
        df = subset(df, df.status .== get_status(:planned))
        df = @set df.metric = ones(Float32, length(df.status))
    elseif metric == "drop"
        df = subset(df, df.status .> get_status(:none))
        df = @set df.metric = df.status .<= get_status(:dropped)
    else
        @assert false
    end
    df
end;

In [ ]:
function write_alpha(
    model::Function,
    medium::String,
    name::String,
    inputs::Vector{String},
)
    alphas = Dict()
    for dataset in ALL_DATASETS
        cols = collect(setdiff(Set(fieldnames(RatingsDataset)), Set([:metric])))
        userids = Set(get_split(dataset, "test_output", medium, [:userid]).userid)
        model_inputs = Dict()
        for x in inputs
            if x in ALL_MEDIUMS
                model_inputs[x] = get_split(dataset, "test_input", x, cols)
            else
                model_inputs[x] = JLD2.load(get_data_path("alphas/$x/alpha.jld2"), dataset)
            end
        end
        preds = model(model_inputs, userids)
        alphas[dataset] = preds
    end
    outdir = mkpath(get_data_path("alphas/$name"))
    JLD2.save("$outdir/alpha.jld2", alphas)
end

function read_alpha(
    dataset::String,
    users::Vector{Int32},
    items::Vector{Int32},
    alpha::String,
)
    alphas = JLD2.load(get_data_path("alphas/$alpha/alpha.jld2"), dataset)
    [alphas[u][i] for (u, i) in zip(users, items)]
end;

In [ ]:
function write_params(params::Dict, outdir::String)
    outdir = mkpath(get_data_path("alphas/$outdir"))
    JLD2.save("$outdir/params.jld2", params; compress = true)
end

function read_params(outdir::String)
    JLD2.load(get_data_path("alphas/$outdir/params.jld2"))
end;